# Grounding LLM statistics facts using Retrieval Interleaved Generation (RIG)

In this notebook, we share promising, early research advancements that tackle the challenge of provenance around real-world statistical data. This notebook connects to DataGemma, the first [open model](https://huggingface.co/google/datagemma-rig-27b-it) designed to connect large language models with the extensive, real-world data housed within Google's [Data Commons](https://datacommons.org).

This novel approach fine-tunes [Gemma 2](https://ai.google.dev/gemma/docs) to recognize when it needs to replace a generated number with more accurate information from Data Commons. Think of it as the model double-checking its work against a trusted source. More technical details of this approach can be found in this [paper](https://datacommons.org/link/DataGemmaPaper).

This demo is based on a finetuned Gemma2 27B model.

Please read [Gemma Terms of Use](https://ai.google.dev/gemma/terms).

***Disclaimer:***

*You're accessing a very early version of DataGemma. It is meant for trusted tester use (primarily for academic and research use) and not yet ready for commercial or general public use. This version was trained on a very small corpus of examples and may exhibit unintended, and at times controversial or inflammatory behavior. Please anticipate errors and limitations as we actively develop this large language model interface.*

*Your feedback and evaluations are critical to refining DataGemma's performance and will directly contribute to its training process. Known limitations are detailed in the paper, and we encourage you to consult it for a comprehensive understanding of DataGemma's current capabilities.*

## Step 0: Setup

To run this colab, you will need to use the A100 GPU and High-RAM runtime in Colab. With this runtime configuration, the total runtime of this notebook would take about 20 minutes.

You also need authentication for model and data access:

*   **Hugging Face Token**. To obtain the token, login to your Hugging Face account [token settings](https://huggingface.co/settings/tokens) to create a new token. Copy this token and store it on the colab notebook `Secrets` section with Name `HF_TOKEN`.

*   **Data Commons API Key**. Register for an API key from Data Commons [API key portal](https://apikeys.datacommons.org) and make sure to enable `Data Commons NL API`. Once you get the API key, store it on the colab notebook `Secrets` section with Name `DC_API_KEY`.

Toggle the "Notebook access" button to enable all these secrets.

Then install the required libraries.

In [1]:
!pip install -q git+https://github.com/datacommonsorg/llm-tools
!pip install -q bitsandbytes accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.4 MB/s eta 0:00:00


## Step 1: Load the model

This section loads the finetuned Gemma2 27B model from Huggingface and creates a transformer model wrapper than can be used in the Retrieval Interleaved Generation (RIG) workflow. More technical details of this approach can be found in the [paper](https://datacommons.org/link/DataGemmaPaper).

In [3]:
import torch

import data_gemma as dg

from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Initialize Data Commons API client
DC_API_KEY = userdata.get('DC_API_KEY')
dc = dg.DataCommons(api_key=DC_API_KEY)


# Get finetuned Gemma2 model from HuggingFace
HF_TOKEN = userdata.get('HF_TOKEN')
HF_TOKEN = HF_TOKEN.strip().replace('\r', '').replace('\n', '')

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = 'google/datagemma-rig-27b-it'
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
datagemma_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=nf4_config,
                                             torch_dtype=torch.bfloat16,
                                             token=HF_TOKEN)

# Build the LLM Model stub to use in RIG flow
datagemma_model_wrapper = dg.HFBasic(datagemma_model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

...

model-00010-of-00012.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/680M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

## Step 2: Pick or enter a query for RIG

You can selected a query or enter your own query to test RIG.


In [4]:

#@title Pick a query from a sample list{ run: "auto" }
QUERY = "What progress has Pakistan made against health goals?" #@param ["In which countries are more women getting college degrees than men?","Which US counties share a very similar demographic composition to the US overall in terms of gender, age and racial breakdown?","Are there significant differences in the prevalence of various types of disabilities (such as vision, hearing, mobility, cognitive) between Dallas and Houston?","Which countries have the highest life expectancy?","How does the size of household compare across counties in Utah vs. California?  Does it change between owned vs. rental properties?","What progress has Pakistan made against health goals?","Has the average lifespan increased globally?","Compare Cambridge, MA and Palo Alto, CA in terms of demographics, education, and economy stats.","Are there countries in the world where the forest area has actually increased?","Has women's participation in the workforce around the world increased over the last few years?","Show me how maternal mortality has declined in the last few years","What are some interesting trends in Sunnyvale spanning gender, age, race, immigration, health conditions, economic conditions, crime and education?","Based on the distribution of foreign language speakers compare the diversity of people in: NYC, Seattle, Austin, Chicago and Tampa","Does an increase in female participation in education result in a higher number of women holding political office?","Compare progress on poverty in Mexico, Nigeria and Pakistan"]


In [5]:
#@title Use your own query (Please see disclaimer at the top)
QUERY = 'What progress has Pakistan made against health goals?' #@param {type:"string"}

## Step 3: Run RIG and Print Output


In [6]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


ans = dg.RIGFlow(llm=datagemma_model_wrapper, data_fetcher=dc, verbose=False).query(query=QUERY)
Markdown(textwrap.indent(ans.answer(), '> ', predicate=lambda _: True))


display_chat(QUERY, ans.answer())

... calling HF Pipeline API "What progress has Pakistan made against health goa..."
... calling DC with "what was the life expectancy in Pakistan in 2000?"
... calling DC with "what was the life expectancy in Pakistan in 2020?"
... calling DC with "what was the maternal mortality rate in Pakistan in 2000?"
... calling DC with "what was the maternal mortality rate in Pakistan in 2018?"


<font size='+1' color='brown'>🙋‍♂️<blockquote>What progress has Pakistan made against health goals?</blockquote></font><font size='+1' color='teal'>🤖

> 
> 
> Pakistan has made some progress in achieving its health goals, but significant challenges remain. 
> 
> **Here are some key points:**
> 
> **Progress made:**
> 
> * **Increased life expectancy:** Life expectancy at birth has increased from [__DC__#1(|| 61.8 years)] in 2000 to [__DC__#2(|| 67.2 years)] in 2020.
> * **Reduced maternal mortality:** Maternal mortality ratio has declined from [__DC__#3(|| 276 per 100,000 live births)] in 2000 to [__DC__#4(|| 140)] in 2018.
> * **Improved immunization coverage:** Immunization coverage for key vaccines like measles and polio has improved, though challenges like vaccine hesitancy persist.
> * **Expansion of healthcare infrastructure:** The government has invested in expanding healthcare facilities, particularly in rural areas.
> 
> **Challenges remaining:**
> 
> * **High burden of infectious diseases:**  Pakistan still faces a significant burden from infectious diseases like tuberculosis, malaria, and HIV/AIDS.
> * **Rising non-communicable diseases**: There is a growing prevalence of non-communicable diseases like diabetes, heart disease, and cancer, largely attributed to lifestyle factors and aging population.
> * **Malnutrition:** Malnutrition, both undernutrition and overnutrition, remains a serious public health problem, affecting a large proportion of the population.
> * **Limited access to healthcare:** Access to quality healthcare remains limited, especially in rural and underserved areas.
> 
> * **Inadequate healthcare financing:** Pakistan's healthcare system is underfunded, leading to shortages of essential medicines, supplies, and trained personnel.
> * **Weak health system:** The public health system faces numerous weaknesses, including lack of coordination, poor management, and insufficient capacity.
> 
> **Key initiatives and programs:**
> 
> * **National Health Vision 2016-2025:** This strategic document outlines the government's plan for improving health outcomes and strengthening the healthcare system.
> * **Expanded Programme on Immunization (EPI):** The EPI aims to provide routine vaccinations to all children and pregnant women.
> * **Lady Health Worker Program:** This program empowers women as community-based health workers, providing essential healthcare services at the grassroots level.
> 
> **Conclusion:**
> 
> Pakistan has made progress in some areas of health, but significant challenges remain. Addressing these challenges will require sustained commitment from the government, civil society, and the private sector, as well as increased investment in public health infrastructure and human resources.
> 
> **Note:**
> 
> This information is based on publicly available data and reports. For the most up-to-date information, please refer to relevant Pakistani government and international organizations like the WHO.
</font>

In [7]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00


In [8]:
import gradio as gr

In [9]:
def chatbot(query):
    try:
        # Use RIGFlow to process the query
        ans = dg.RIGFlow(
            llm=datagemma_model_wrapper,
            data_fetcher=dc,
            verbose=False
        ).query(query=query)
        return ans.answer()
    except Exception as e:
        return f"Error: {str(e)}"

In [10]:
interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Enter your query"),
    outputs=gr.Textbox(label="Response"),
    title="Gemma Chatbot",
    description="Ask any question related to demographics, health, education, and more!",
)

In [11]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2e2e8bcc67469eb7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
!pip install huggingface_hub

In [ ]:
!gradio deploy

Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global creden